# 📊 Analyse et Visualisation des Données des Cours

Ce notebook présente une analyse complète et interactive du dataset des cours (`final_data.csv`).
Nous utiliserons **Plotly** pour des visualisations dynamiques et esthétiques afin de mieux comprendre la distribution des données pour notre système de recommandation.

### Objectifs de l'analyse :
1. Distribution des cours par catégorie
2. Analyse des notes (Ratings)
3. Relation entre popularité et notes
4. Top 10 des partenaires (institutions)
5. Distribution des niveaux de difficulté
6. Durée des cours par catégorie
7. Répartition par domaine source

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')

# Configuration du thème Plotly
px.defaults.template = "plotly_dark"
px.defaults.color_continuous_scale = px.colors.sequential.Viridis

## 1. Chargement et Aperçu des Données

In [ ]:
# Chargement du fichier CSV
try:
    # Chemin relatif depuis le dossier notebooks/
    df = pd.read_csv('../final_data/final_data.csv')
    print("✅ Données chargées avec succès !")
except FileNotFoundError:
    try:
        # Fallback 1: Si exécuté depuis la racine
        df = pd.read_csv('final_data/final_data.csv')
        print("✅ Données chargées avec succès (chemin racine) !")
    except FileNotFoundError:
        # Fallback 2: Chemin absolu ou autre structure
        print("❌ Fichier non trouvé. Vérifiez le chemin.")
        # df = pd.read_csv('chemin/vers/final_data.csv')

# Conversion des colonnes si nécessaire
if 'scraped_at' in df.columns:
    df['scraped_at'] = pd.to_datetime(df['scraped_at'])

# Affichage des premières lignes
df.head()

In [ ]:
# Informations générales
df.info()

## 2. Distribution des Catégories
Quelles sont les thématiques les plus représentées dans notre catalogue ?

In [ ]:
category_counts = df['category'].value_counts().reset_index()
category_counts.columns = ['Category', 'Count']

fig = px.bar(category_counts, 
             x='Count', 
             y='Category', 
             orientation='h',
             title='<b>Distribution des Cours par Catégorie</b>',
             text='Count',
             color='Count', 
             color_continuous_scale='Viridis')

fig.update_layout(yaxis={'categoryorder':'total ascending'}, showlegend=False)
fig.show()

## 3. Analyse des Notes (Ratings)
Analyse de la distribution des notes attribuées aux cours.

In [ ]:
fig = px.histogram(df, 
                   x='rating', 
                   nbins=20, 
                   title='<b>Distribution des Notes des Cours</b>',
                   color_discrete_sequence=['#00CC96'],
                   marginals='box', 
                   hover_data=df.columns)

fig.update_layout(bargap=0.1)
fig.show()

## 4. Relation Popularité vs Notes
Y a-t-il une corrélation entre la note d'un cours et son score de popularité ?

In [ ]:
fig = px.scatter(df, 
                 x='rating', 
                 y='popularity_score', 
                 color='category',
                 size='num_ratings', 
                 hover_name='title',
                 title='<b>Popularité vs Notes</b>',
                 log_y=True, 
                 template='plotly_dark')

fig.show()

## 5. Top 10 des Partenaires
Quelles institutions proposent le plus de cours ?

In [ ]:
partner_counts = df['partner'].value_counts().nlargest(10).reset_index()
partner_counts.columns = ['Partner', 'Count']

fig = px.bar(partner_counts, 
             x='Partner', 
             y='Count', 
             title='<b>Top 10 des Institutions Partenaires</b>',
             color='Count',
             text='Count')

fig.update_layout(xaxis_tickangle=-45)
fig.show()

## 6. Distribution des Niveaux
Répartition des niveaux de difficulté des cours.

In [ ]:
level_counts = df['level_enc'].value_counts().reset_index()
level_counts.columns = ['Level Code', 'Count']

fig = px.pie(level_counts, 
             values='Count', 
             names='Level Code',
             title='<b>Distribution des Niveaux de Cours</b>',
             hole=0.4)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## 7. Durée des Cours par Catégorie
Quelle est la durée typique des cours selon leur thématique ?

In [ ]:
fig = px.box(df, 
             x='category', 
             y='duration_hours', 
             color='category',
             title='<b>Distribution de la Durée des Cours par Catégorie</b>',
             points='outliers')

fig.update_layout(xaxis_tickangle=-45, showlegend=False)
fig.show()

## 8. Répartition par Domaine Source
D'où proviennent les données ?

In [ ]:
source_counts = df['source_domain'].value_counts().reset_index()
source_counts.columns = ['Source', 'Count']

fig = px.pie(source_counts, 
             values='Count', 
             names='Source',
             title='<b>Répartition des Cours par Source</b>',
             color_discrete_sequence=px.colors.qualitative.Pastel)

fig.show()

## Conclusion et Insights

Cette analyse nous permet de dégager plusieurs points clés pour notre système de recommandation :
1. **Dominance des catégories** : Certaines catégories sont très représentées, ce qui nécessitera peut-être un équilibrage lors des recommandations.
2. **Qualité des données** : La distribution des notes montre la tendance générale de satisfaction.
3. **Engagement** : La relation popularité/notes aide à identifier les "pépites" (cours très notés et populaires) à mettre en avant.

Ces visualisations serviront de base pour affiner nos algorithmes de filtrage et de ranking.